In [ ]:
!pip install torch torch-geometric pandas numpy matplotlib fastdtw scipy scikit-learn pyarrow

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import from_scipy_sparse_matrix
from sklearn.neighbors import kneighbors_graph
from scipy.spatial.distance import pdist, squareform
from fastdtw import fastdtw
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import uuid
import asyncio
import platform
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

torch.manual_seed(2025)
np.random.seed(2025)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

required_packages = ['torch', 'torch_geometric', 'pandas', 'numpy', 'matplotlib', 'fastdtw', 'scipy', 'scikit-learn', 'pyarrow']
try:
    for pkg in required_packages:
        __import__(pkg.replace('-', '_'))
except ImportError as e:
    raise ImportError(f"Missing package. Install with: pip install {' '.join(required_packages)}")

base_path = './BiasCorrectionData/'
os.makedirs(base_path, exist_ok=True)
model_input_path = os.path.join(base_path, 'model_input_all_features_new.parquet')
satellite_data_path = os.path.join(base_path, 'satellite_data_processed_thailand_time.parquet')
submission_file = os.path.join(base_path, 'submission.csv')

In [ ]:
def load_processed_data(model_input_path, satellite_data_path):
    """Load and merge processed data files."""
    try:
        model_df = pd.read_parquet(model_input_path)
        satellite_df = pd.read_parquet(satellite_data_path)
        merged_df = pd.merge(model_df, satellite_df,
                            left_on=['datetime', 'Station'],
                            right_on=['datetime', 'Station'],
                            how='left')
        merged_df['timestamp'] = pd.to_datetime(merged_df['datetime'])
        merged_df.drop(columns=['datetime'], inplace=True)
        if merged_df.empty:
            raise ValueError("Merged dataframe is empty after joining.")
        return merged_df
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Data file not found: {e}")
    except Exception as e:
        raise Exception(f"Error loading data: {e}")

In [ ]:
def preprocess_features(df):
    """Preprocess features including scaling and handling cyclical features."""
    numerical_features = ['GSMap', 'HII', 'Humidity', 'Pressure', 'Temperature',
                         'temperature_2m_celsius', 'dewpoint_temperature_2m_celsius',
                         'relative_humidity', 'wind_speed_ms', 'surface_pressure_hpa',
                         'sea_level_pressure_hpa', 'total_precipitation_mm', 'solar_radiation_wm2']
    cyclical_features = ['hour_sin', 'hour_cos', 'month_sin', 'month_cos',
                        'dayofyear_sin', 'dayofyear_cos', 'weekday_sin', 'weekday_cos']
    lag_features = [col for col in df.columns if 'lag' in col or 'roll' in col]
    bias_features = [col for col in df.columns if 'bias' in col]
    neighbor_features = [col for col in df.columns if 'nei_' in col]

    scaler = StandardScaler()
    for feature_group in [numerical_features, lag_features, bias_features, neighbor_features]:
        if feature_group and any(col in df.columns for col in feature_group):
            valid_cols = [col for col in feature_group if col in df.columns]
            df[valid_cols] = scaler.fit_transform(df[valid_cols])

    df.fillna(0, inplace=True)
    return df, scaler

In [ ]:
def create_spatio_temporal_dataset(df, seq_len=7, target_col='precipitation'):
    """Create spatio-temporal dataset from processed dataframe."""
    feature_cols = [col for col in df.columns if col not in ['Station', 'timestamp', target_col]]
    stations = df['Station'].unique()
    timestamps = df['timestamp'].unique()
    num_stations = len(stations)
    num_features = len(feature_cols)
    num_samples = len(timestamps) - seq_len + 1

    if num_samples <= 0:
        raise ValueError(f"Not enough timestamps ({len(timestamps)}) for sequence length {seq_len}")

    X = np.zeros((num_samples, seq_len, num_stations, num_features), dtype=np.float32)
    y = np.zeros((num_samples, num_stations, 1), dtype=np.float32)

    for i in tqdm(range(num_samples), desc="Creating dataset"):
        seq_timestamps = timestamps[i:i + seq_len]
        seq_data = df[df['timestamp'].isin(seq_timestamps)]
        for t, timestamp in enumerate(seq_timestamps):
            t_data = seq_data[seq_data['timestamp'] == timestamp].set_index('Station').reindex(stations)
            X[i, t] = t_data[feature_cols].values
            if t == seq_len - 1:
                y[i] = t_data[target_col].values.reshape(-1, 1)

    return X, y, stations, feature_cols

In [ ]:
def create_graph_structure(coords_df, data_df, feature_cols, k=5):
    """Create graph structure from coordinates and data."""
    coords = coords_df[['lat', 'long']].values
    num_stations = len(coords)
    station_ids = coords_df['Station'].values

    distance_matrix = squareform(pdist(coords, 'euclidean'))
    binary_adj = kneighbors_graph(coords, n_neighbors=k, mode='connectivity', include_self=False).toarray()
    distance_adj = np.exp(-distance_matrix / (distance_matrix.std() + 1e-8))
    np.fill_diagonal(distance_adj, 0)

    corr_adj = np.zeros((num_stations, num_stations))
    for i in range(num_stations):
        for j in range(i + 1, num_stations):
            ts1 = data_df[data_df['Station'] == station_ids[i]]['precipitation'].values
            ts2 = data_df[data_df['Station'] == station_ids[j]]['precipitation'].values
            if len(ts1) > 0 and len(ts2) > 0 and len(ts1) == len(ts2):
                corr = np.corrcoef(ts1, ts2)[0, 1]
                corr_adj[i, j] = corr_adj[j, i] = max(corr, 0) if not np.isnan(corr) else 0

    dtw_adj = np.zeros((num_stations, num_stations))
    max_dtw = 0
    for i in range(num_stations):
        for j in range(i + 1, num_stations):
            ts1 = data_df[data_df['Station'] == station_ids[i]]['precipitation'].values
            ts2 = data_df[data_df['Station'] == station_ids[j]]['precipitation'].values
            if len(ts1) > 0 and len(ts2) > 0:
                distance, _ = fastdtw(ts1, ts2)
                dtw_adj[i, j] = dtw_adj[j, i] = distance
                max_dtw = max(max_dtw, distance)
    dtw_adj = 1 - (dtw_adj / (max_dtw + 1e-8))

    feature_matrix = data_df.groupby('Station')[feature_cols].mean().values
    feature_sim = np.exp(-squareform(pdist(feature_matrix, 'cosine')))
    np.fill_diagonal(feature_sim, 0)

    edge_index, _ = from_scipy_sparse_matrix(binary_adj)
    edge_attr = np.stack([distance_adj[binary_adj > 0], corr_adj[binary_adj > 0],
                          dtw_adj[binary_adj > 0], feature_sim[binary_adj > 0]], axis=-1)

    return edge_index.to(device), torch.FloatTensor(edge_attr).to(device)

In [ ]:
class EnhancedSTGNNLayer(MessagePassing):
    def __init__(self, node_dim, edge_dim, hidden_dim):
        super(EnhancedSTGNNLayer, self).__init__(aggr='mean')
        self.edge_attention = nn.Sequential(
            nn.Linear(edge_dim + node_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        self.message_mlp = nn.Sequential(
            nn.Linear(node_dim + edge_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(0.2)
        )
        self.update_mlp = nn.Sequential(
            nn.Linear(node_dim + hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, node_dim),
            nn.Dropout(0.2)
        )

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr):
        edge_input = torch.cat([x_i, x_j, edge_attr], dim=-1)
        attention = self.edge_attention(edge_input)
        message_input = torch.cat([x_j, edge_attr], dim=-1)
        message = self.message_mlp(message_input)
        return message * attention

    def update(self, aggr_out, x):
        update_input = torch.cat([x, aggr_out], dim=-1)
        return self.update_mlp(update_input)

In [ ]:
class EnhancedTemporalAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=4):
        super(EnhancedTemporalAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.scale = self.head_dim ** -0.5
        self.out_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        Q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        scores = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        attn_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attn_weights, V).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        return self.out_proj(output)

In [ ]:
class EnhancedBinarySTGNN(nn.Module):
    def __init__(self, node_dim, edge_dim, hidden_dim, num_layers, num_columns):
        super(EnhancedBinarySTGNN, self).__init__()
        self.input_proj = nn.Linear(node_dim, hidden_dim)
        self.spatial_layers = nn.ModuleList([
            EnhancedSTGNNLayer(hidden_dim, edge_dim, hidden_dim) for _ in range(num_layers)
        ])
        self.temporal_attention = EnhancedTemporalAttention(hidden_dim)
        self.temporal_norm = nn.LayerNorm(hidden_dim)
        self.binary_heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim // 2),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(hidden_dim // 2, 1)
            ) for _ in range(num_columns)
        ])

    def forward(self, x, edge_index, edge_attr):
        batch_size, seq_len, num_nodes, _ = x.shape
        x = self.input_proj(x)
        spatial_outputs = []
        for t in range(seq_len):
            x_t = x[:, t]
            for layer in self.spatial_layers:
                x_t = layer(x_t, edge_index, edge_attr)
            spatial_outputs.append(x_t.unsqueeze(1))
        x = torch.cat(spatial_outputs, dim=1).permute(0, 2, 1, 3).reshape(batch_size * num_nodes, seq_len, -1)
        x = self.temporal_attention(x)
        x = self.temporal_norm(x)
        features = x[:, -1, :].reshape(batch_size, num_nodes, -1)
        binary_predictions = [head(features) for head in self.binary_heads]
        return torch.cat(binary_predictions, dim=-1), features

In [ ]:
class MetaKAN(nn.Module):
    def __init__(self, input_dim, num_columns, hidden_dims=[64, 32]):
        super(MetaKAN, self).__init__()
        self.hypernetwork = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.kan_networks = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dims[0]),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_dims[0], hidden_dims[1]),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(hidden_dims[1], 1)
            ) for _ in range(num_columns)
        ])

    def forward(self, x, binary_mask):
        batch_size, num_nodes, _ = x.shape
        value_predictions = torch.zeros_like(binary_mask, device=x.device)
        for col_idx, kan_net in enumerate(self.kan_networks):
            col_mask = binary_mask[:, :, col_idx]
            nonzero_positions = col_mask > 0.5
            if nonzero_positions.any():
                masked_features = x[nonzero_positions]
                predicted_values = kan_net(masked_features)
                predicted_values = torch.clamp(predicted_values, 0, 99)
                value_predictions[:, :, col_idx][nonzero_positions] = predicted_values.squeeze(-1)
        return value_predictions

In [ ]:
class GPHead(nn.Module):
    def __init__(self, input_dim, num_columns, hidden_dims=[64, 32]):
        super(GPHead, self).__init__()
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dims[0]),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(hidden_dims[0], hidden_dims[1]),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(hidden_dims[1], 2)
            ) for _ in range(num_columns)
        ])

    def forward(self, x, binary_mask, num_samples=10):
        batch_size, num_nodes, _ = x.shape
        mean_predictions = torch.zeros_like(binary_mask, device=x.device)
        var_predictions = torch.zeros_like(binary_mask, device=x.device)
        self.train()
        for col_idx, layer in enumerate(self.layers):
            col_mask = binary_mask[:, :, col_idx]
            nonzero_positions = col_mask > 0.5
            if nonzero_positions.any():
                masked_features = x[nonzero_positions]
                samples = torch.stack([layer(masked_features) for _ in range(num_samples)], dim=0)
                mean = samples[:, :, 0].mean(dim=0)
                log_variance = samples[:, :, 1].mean(dim=0)
                variance = torch.exp(log_variance.clamp(min=-10, max=10))
                mean = torch.clamp(mean, 0, 99)
                mean_predictions[:, :, col_idx][nonzero_positions] = mean
                var_predictions[:, :, col_idx][nonzero_positions] = variance
        return mean_predictions, var_predictions

In [ ]:
class EnhancedTwoStageModel(nn.Module):
    def __init__(self, node_dim, edge_dim, hidden_dim, num_layers, num_columns):
        super(EnhancedTwoStageModel, self).__init__()
        self.binary_model = EnhancedBinarySTGNN(node_dim, edge_dim, hidden_dim, num_layers, num_columns)
        self.value_model = MetaKAN(hidden_dim, num_columns)
        self.gp_head = GPHead(hidden_dim, num_columns)
        self.feature_enhancer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim * 2, hidden_dim)
        )

    def forward(self, x, edge_index, edge_attr, num_samples=10):
        binary_logits, features = self.binary_model(x, edge_index, edge_attr)
        binary_probs = torch.sigmoid(binary_logits)
        binary_predictions = (binary_probs > 0.5).float()
        enhanced_features = self.feature_enhancer(features)
        value_predictions = self.value_model(enhanced_features, binary_predictions)
        mean_predictions, var_predictions = self.gp_head(enhanced_features, binary_predictions, num_samples)
        return binary_logits, binary_predictions, mean_predictions, var_predictions, enhanced_features

In [ ]:
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x, edge_index, edge_attr, num_samples=10):
        outputs = [model(x, edge_index, edge_attr, num_samples) for model in self.models]
        binary_logits = torch.mean(torch.stack([out[0] for out in outputs]), dim=0)
        binary_predictions = (torch.sigmoid(binary_logits) > 0.5).float()
        mean_predictions = torch.mean(torch.stack([out[2] for out in outputs]), dim=0)
        var_predictions = torch.mean(torch.stack([out[3] for out in outputs]), dim=0)
        return binary_logits, binary_predictions, mean_predictions, var_predictions

In [ ]:
def train_model(model, train_loader, edge_index, edge_attr, optimizer, num_epochs, device):
    binary_criterion = nn.BCEWithLogitsLoss()
    value_criterion = nn.MSELoss()
    metrics = {'binary_loss': [], 'value_loss': [], 'binary_acc': [], 'pbiasabs': []}

    for epoch in range(num_epochs):
        model.train()
        epoch_metrics = {'binary_loss': 0, 'value_loss': 0, 'binary_acc': 0, 'pbiasabs': 0}
        for batch_X, batch_binary_y, batch_value_y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch_X, batch_binary_y, batch_value_y = [x.to(device) for x in [batch_X, batch_binary_y, batch_value_y]]
            optimizer.zero_grad()
            binary_logits, binary_predictions, mean_predictions, _, _ = model(batch_X, edge_index, edge_attr)

            binary_loss = binary_criterion(binary_logits, batch_binary_y)
            value_mask = batch_binary_y > 0.5
            value_loss = torch.tensor(0.0, device=device)
            pbiasabs = torch.tensor(0.0, device=device)
            if value_mask.any():
                masked_value_pred = mean_predictions[value_mask]
                masked_value_true = batch_value_y[value_mask]
                value_loss = value_criterion(masked_value_pred, masked_value_true)
                pbiasabs = 100 * torch.abs(masked_value_pred - masked_value_true).sum() / (masked_value_true.sum() + 1e-8)

            total_loss = binary_loss + 0.5 * value_loss
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_metrics['binary_loss'] += binary_loss.item()
            epoch_metrics['value_loss'] += value_loss.item()
            epoch_metrics['binary_acc'] += accuracy_score(batch_binary_y.cpu().numpy().flatten(),
                                                         binary_predictions.cpu().numpy().flatten())
            epoch_metrics['pbiasabs'] += pbiasabs.item()

        for key in epoch_metrics:
            metrics[key].append(epoch_metrics[key] / len(train_loader))
        print(f"Epoch {epoch+1}: Binary Loss: {metrics['binary_loss'][-1]:.4f}, "
              f"Value Loss: {metrics['value_loss'][-1]:.4f}, "
              f"Accuracy: {metrics['binary_acc'][-1]:.4f}, PBIASabs: {metrics['pbiasabs'][-1]:.4f}%")

    return model, metrics

In [ ]:
def evaluate_model(model, loader, edge_index, edge_attr, device, num_samples=10):
    model.eval()
    binary_correct = 0
    total_samples = 0
    value_abs_error = 0
    value_sum = 0
    constraint_violations = 0

    with torch.no_grad():
        for batch_X, batch_binary_y, batch_value_y in loader:
            batch_X, batch_binary_y, batch_value_y = [x.to(device) for x in [batch_X, batch_binary_y, batch_value_y]]
            binary_logits, binary_predictions, mean_predictions, _, _ = model(batch_X, edge_index, edge_attr, num_samples)
            binary_correct += (binary_predictions == batch_binary_y).float().sum().item()
            total_samples += batch_binary_y.numel()
            value_mask = batch_binary_y > 0.5
            if value_mask.any():
                masked_value_pred = mean_predictions[value_mask]
                masked_value_true = batch_value_y[value_mask]
                value_abs_error += torch.abs(masked_value_pred - masked_value_true).sum().item()
                value_sum += masked_value_true.sum().item()
                constraint_violations += ((masked_value_pred > 99) | (masked_value_pred < 0)).sum().item()

    binary_accuracy = binary_correct / (total_samples + 1e-8)
    pbiasabs = 100 * value_abs_error / (value_sum + 1e-8) if value_sum > 0 else 0
    violation_rate = constraint_violations / (total_samples + 1e-8)

    print(f"\nModel Evaluation:")
    print(f"Binary Accuracy: {binary_accuracy:.4f}")
    print(f"PBIASabs: {pbiasabs:.4f}%")
    print(f"Constraint Violation Rate: {violation_rate:.6f}")

    return binary_accuracy, pbiasabs, violation_rate

In [ ]:
def generate_submission(model, test_loader, edge_index, edge_attr, device, stations, output_file, num_samples=10):
    model.eval()
    predictions = []
    variances = []
    timestamps = pd.date_range(start='2021-01-01', periods=len(test_loader.dataset), freq='D')

    with torch.no_grad():
        for batch_X in test_loader:
            batch_X = batch_X[0].to(device)
            _, binary_predictions, mean_predictions, var_predictions, _ = model(batch_X, edge_index, edge_attr, num_samples)
            output = mean_predictions * binary_predictions
            predictions.append(output.cpu().numpy())
            variances.append(var_predictions.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    variances = np.concatenate(variances, axis=0)
    submission_data = []
    for i in range(predictions.shape[0]):
        for s, station in enumerate(stations):
            submission_data.append({
                'Station': station,
                'timestamp': timestamps[i],
                'precip': float(predictions[i, s, 0]),
                'variance': float(variances[i, s, 0])
            })

    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(output_file, index=False)
    print(f"Submission file saved to {output_file}")

In [ ]:
def save_model(model, metrics, binary_acc, pbiasabs, violation_rate):
    torch.save({
        'model_state_dict': model.state_dict(),
        'hyperparameters': {
            'node_dim': node_dim,
            'edge_dim': edge_dim,
            'hidden_dim': hidden_dim,
            'num_layers': num_layers,
            'num_columns': num_columns,
            'num_ensemble_models': num_ensemble_models
        },
        'training_history': metrics,
        'evaluation_results': {
            'binary_accuracy': binary_acc,
            'pbiasabs': pbiasabs,
            'violation_rate': violation_rate
        }
    }, os.path.join(base_path, 'enhanced_two_stage_model.pth'))
    print("Model saved successfully!")

In [ ]:
node_dim = None
edge_dim = 4
hidden_dim = 128
num_layers = 3
num_columns = 1
seq_len = 7
batch_size = 32
num_epochs = 100
num_ensemble_models = 3
num_gp_samples = 10
learning_rate = 0.001

In [ ]:
def main():
    try:
        print("Loading and preprocessing data...")
        merged_df = load_processed_data(model_input_path, satellite_data_path)
        processed_df, scaler = preprocess_features(merged_df)
        X, y, stations, feature_cols = create_spatio_temporal_dataset(processed_df, seq_len)

        coords_df = processed_df[['Station', 'lat', 'long']].drop_duplicates()
        edge_index, edge_attr = create_graph_structure(coords_df, processed_df, feature_cols)


        train_size = int(0.8 * len(X))
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        binary_y_train = (y_train > 0).astype(np.float32)
        binary_y_test = (y_test > 0).astype(np.float32)


        train_dataset = torch.utils.data.TensorDataset(
            torch.FloatTensor(X_train), torch.FloatTensor(binary_y_train), torch.FloatTensor(y_train))
        test_dataset = torch.utils.data.TensorDataset(
            torch.FloatTensor(X_test), torch.FloatTensor(binary_y_test), torch.FloatTensor(y_test))


        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


        node_dim = X_train.shape[-1]
        models = [EnhancedTwoStageModel(node_dim, edge_dim, hidden_dim, num_layers, num_columns).to(device)
                  for _ in range(num_ensemble_models)]
        ensemble_model = EnsembleModel(models).to(device)


        print("Training model...")
        optimizer = torch.optim.AdamW(ensemble_model.parameters(), lr=learning_rate, weight_decay=1e-4)
        ensemble_model, metrics = train_model(ensemble_model, train_loader, edge_index, edge_attr,
                                             optimizer, num_epochs, device)

        print("Evaluating model...")
        binary_acc, pbiasabs, violation_rate = evaluate_model(ensemble_model, test_loader, edge_index, edge_attr, device)


        print("Generating submission...")
        generate_submission(ensemble_model, test_loader, edge_index, edge_attr, device, stations, submission_file)


        print("Saving model...")
        save_model(ensemble_model, metrics, binary_acc, pbiasabs, violation_rate)

        print("Training complete!")

    except Exception as e:
        print(f"Error: {e}")
        raise

if platform.system() == "Emscripten":
    asyncio.run(main())
else:
    main()